# Home assigment 2

Please give your name below:

In [ ]:
name='Robin Vestling'

## Exercise 2

Now you will put together the pieces you learnt in previous labs in order to write a continuous energy Monte Carlo particle transport simulation of a Uranium sphere! Your goal is to estimate the k-effective and to try to find the critical radius of a U-235 sphere. 

Your assumptions:

- The geometry is a sphere
- The sphere is made of pure U-235.
- You only care about the following reactions: capture, fission and elastic scattering 
- All scattering is isotropic in the CM frame.
- Neutrons emerge isotropically from fission.
- You received the pointwise cross sections and the energy dependent nubar data in the /data folder.
- You can neglect that in a fission event the number of generated neutrons is stochastic, and assume that always nubar neutrons are created.
- For the prompt neutron energies you can sample the Watt-distribution (see lecture notes, or Datalab 4)
- You do not need to track time (thus all neutrons can be considered to be prompt)
- Initially launch neutrons from the center of the sphere, then store fission sites, and later sample the new fission sites from this "bank".

Your tasks:

1. Plot the the cross section data and the nubar data.
2. Complete the support functions given below and the function `run()` in order to estimate the k-eigenvalue of a sphere with a continous energy 3D Monte Carlo particle transport simulation. The support functions are the ones which you saw in Datalab 5b (for example the direction transformations, elastic scattering etc.). Some of these functions you will need to update (eg. for the reaction type sampler, include fission). You can include other input parameters and set default values if you feel needed. For each neutron generation estimate the k-eigenvalue based on the initial number of neutrons and the new neutrons after the generation (as we did in Datalab 5a).
3. Modify the return list in order to return and plot other data
    - Plot the k-eigenvalue estimate vs the generation number
    - Plot how the estimated mean k-eigenvalue converges. (use such figures to argue about reasonable values for `NGEN`, `NPG`, `NSKIP`). 
4. Investigate how the k-eigenvalue depends on the radius of the sphere. Visualize this with matplotlib.
5. Find the critical radius. You can do this either with manual trial and error, or use an optimization method.


Hints: in this exercise you have to merge your knowledge from datalab 5a (ie. batchwise estimation of k-effective) and from datalab 5b (ie. tracking neutrons). If you are not sure about the validity of your results you can compare your findings with the values of critical radii from [Wikipedia](https://en.wikipedia.org/wiki/Critical_mass). Try to have similar order of magnitude results.

To be fair, in a real MC criticality calculation, the initial number of neutrons per cycle also fluctuates, and the k-eigenvalue is calculated with some power iteration. In that case some care needs to be taken to renormalize the number of events to be placed in the bank, in order to have more or less the same amount of starting neutrons in each batch, otherwise sub and supercritical systems would be problematic to be simulated (here p200-225 gives some details on that: https://mcnp.lanl.gov/pdf_files/la-ur-16-29043.pdf). You don't need to worry about these. We are satisfied with a simpler approach. Rather you will initiate the same amount of neutrons in each cycle, regardless how many were produced before, and we place every fission site into the bank, and sample the locations from that. We also do not require an initial guess for the k-eigenvalue (as you can see in the link for the power iteration based method, an initial guess is needed). 

In the first few cycles when we launch neutrons only from the center, we will probably underestimate leakage, so the estimates of $k$ will be biased. Therefore NSKIP just means that the first NSKIP number of cycle estimates of the k-effective should not be taken into account when calculating the mean of the k-effective, since the spatial distribution of the fission source is still biased by our original source location, and not spread yet throughout the geometry. Actually for this simple geometry NSKIP plays a less important role, so if you are not certain about what it is, feel free to ignore it. 

Try not to overcomplicate the exercise. The function `run()` with docstrings and comments can be written in less than 80 lines of code. Below we collected all the supporting functions from Datalab 4 and 5, which you will need to use. Some of them you need to update or finish first. We also loaded the nuclear data.

Also, ideally the computation should not be too slow. Test first with small NGEN and NPG values (eg. 100 for both). This should already provide decent accuracy. I tested that on an older laptop this many batches and particles can be run within a minute without any vectorization. If you experience that your computation is much longer, there might be a mistake.

In [7]:
import numpy as np
import random
import matplotlib.pyplot as plt

Es,xss=np.loadtxt('u235el.dat',skiprows=2).transpose()
Ec,xsc=np.loadtxt('u235cap.dat',skiprows=2).transpose()
Ef,xsf=np.loadtxt('u235fiss.dat',skiprows=2).transpose()

Enu,nubar=np.loadtxt('u235nubar.dat',skiprows=2).transpose()


density = 19.1 #g/cm3
A = 235

#TODO : get the macroscopic cross section
Numdens = density * 6.022E23 / A 

MXSc= xsc * Numdens*1e-24
MXSs= xss * Numdens*1e-24
MXSf= xsf * Numdens*1e-24
##### SUPPORT functions
def distanceToCollision(SigT,N=1):
    x=np.random.uniform(0,1,N)
    return -np.log(x)/SigT

def reactionType(SigS,SigC,SigF,SigT):
    #TODO: include the fission cross section!

    x=np.random.uniform(0,1)
    if x < SigS/SigT:
        return 'scatter'
    elif SigS/SigT < x < (SigS+SigC)/SigT:
        return 'capture'
    else:
        return 'fission'

def elasticScatter(E):
    muC=np.random.uniform(-1,1)
    thetaC=np.arccos(muC)
    alpha = ((A - 1) / (A + 1)) ** 2
    E=(((1+alpha)+(1-alpha)*muC)/2)*E
    thetaL=np.arctan2(np.sin(thetaC),((1/A)+muC))
    muL=np.cos(thetaL)
    return E, muL

def randomDir():
    mu=np.random.uniform(-1,1)
    theta=np.arccos(mu)
    phi=np.random.uniform(0,2*np.pi)

    u=np.sin(theta)*np.cos(phi)
    v=np.sin(theta)*np.sin(phi)
    w=np.cos(theta)
    return np.array([u,v,w])

def transformDir(u,v,w,mu):
    """
    transform coordinates according to openMC documentation.
    
    Parameters
    ----------
    u : float
        Old x-direction
    v : float
        Old y-direction
    w : float
        Old z-direction
    mu : float
        Lab cosine of scattering angle
    """
    phi=np.random.uniform(0,2*np.pi)
    un=mu*u+(np.sqrt(1-mu**2)*(u*w*np.cos(phi)-v*np.sin(phi)))/(np.sqrt(1-w**2))
    vn=mu*v+(np.sqrt(1-mu**2)*(v*w*np.cos(phi)+u*np.sin(phi)))/(np.sqrt(1-w**2))
    wn=mu*w-np.sqrt(1-mu**2)*np.sqrt(1-w**2)*np.cos(phi)
    return np.array([un,vn,wn])

def watt(x): 
    """
    Function to return the Watt distribution

    Parameters
    ----------
    x : float
        Energy in MeV
    """
    C1 = 0.453
    C2 = 0.965
    C3 = 2.29
    return C1*np.exp(-x/C2)*np.sinh(np.sqrt(C3*x))

def wattrnd(N):
    """
    Function to return energies sampled from the Watt-distribution.

    Parameters
    ----------
    N : int
        Number of samples needed
    """
    #TODO: you have to complete this function. You can use the rejection method
    # which we used in Datalab4. Just now the dots we plotted as "accepted" you 
    # can keep and return.
    energies = []
    maxW=np.max(watt(Enu))
    for _ in range(100):
        xi=np.random.uniform(0,10)
        yi=np.random.uniform(0,maxW)#complete the line
        if yi<watt(xi):
            energies.append(yi)
            
    # NOTE: take care of the energy units! Make sure, that the energy unit you use
    # here matches the energy unit used in your simulation!
    return energies




In [11]:
def run(R,NGEN,NPG,NSKIP):
    """Function to perform a criticality calculation in a U-235 sphere.
    
    Parameters
    ----------
    R : float
        Radius of the sphere
    NGEN : int
        Number of neutron generations
    NPG : int
        Number of neutrons per generation
    NSKIP : int
        Number of inactive generations which will not be taken into account for estimating the k-eigenvalue
    
    Returns
    -------
    keff : float
        The estimated mean k-eigenvalue of the system
    kstd : float
        The standard deviation of the estimated k-eigenvalue
    """
    k_eff = []
    fission_sites = []

    for gen in range(NGEN):
        new_fission_sites = []
        neutrons = [randomDir() for _ in range(NPG)]  # Initialize directions

        for _ in range(NPG):
            position = np.zeros(3)  # Start from the center
            neutron = randomDir()  # Initial direction of the neutron

            while np.linalg.norm(position) < R:
                SigT = (MXSc[np.searchsorted(Es, np.linalg.norm(neutron))] +
                        MXSs[np.searchsorted(Es, np.linalg.norm(neutron))] +
                        MXSf[np.searchsorted(Es, np.linalg.norm(neutron))])
                distance = distanceToCollision(SigT)
                
                if np.linalg.norm(position + distance * neutron) >= R:
                    break

                # Update position
                position += distance * np.array(neutron)  # Ensure neutron is a numpy array

                SigS = MXSs[np.searchsorted(Es, np.linalg.norm(neutron))]
                SigC = MXSc[np.searchsorted(Es, np.linalg.norm(neutron))]
                SigF = MXSf[np.searchsorted(Es, np.linalg.norm(neutron))]
                reaction = reactionType(SigS, SigC, SigF, SigT)

                if reaction == 'scatter':
                    E = np.linalg.norm(neutron)  # Current neutron energy
                    new_energy, new_direction = elasticScatter(E)
                    neutron = transformDir(neutron[0], neutron[1], neutron[2], randomDir())
                elif reaction == 'capture':
                    break
                elif reaction == 'fission':
                    # Interpolate nubar based on neutron energy
                    neutron_energy = np.linalg.norm(neutron)
                    nubar_interp = np.interp(neutron_energy, Enu, nubar)

                    fission_sites.append(position.copy())
                    # Generate new neutrons based on nubar
                    for _ in range(int(np.round(nubar_interp))):
                        new_fission_sites.append(np.random.uniform(-R, R, 3))

        if gen >= NSKIP:
            k_eff.append(len(new_fission_sites) / NPG)

    k_eff = np.array(k_eff)
    keff = np.mean(k_eff)
    kstd = np.std(k_eff)
    return keff, kstd

R = 5.0  # cm, example radius
NGEN = 100
NPG = 100
NSKIP = 10

keff, kstd = run(R, NGEN, NPG, NSKIP)
print(f"Estimated k-effective: {keff:.4f} ± {kstd:.4f}")

NameError: name 'alpha' is not defined